In [1]:
from vpython import *
from numpy import pi, array, sign
import math

# These functions calculate the magnus forces for each axis
def magnusx(vy, vz):
    return 0.5*cd*rho*area*rad*(wy*vz - wz*vy)/S

def magnusy(vx, vz):
    return 0.5*cd*rho*area*rad*(wz*vx - wx*vz)/S

def magnusz(vx, vy):
    return 0.5*cd*rho*area*rad*(wx*vy - wy*vx)/S

# This function solves df/dt = f(r,t) for Magnus force
def fM(r,t):
    
    # unpack variables
    x = r[0]
    y = r[1]
    z = r[2]
    vx = r[3]
    vy = r[4]
    vz = r[5]
    v = sqrt(vx**2 + vy**2 + vz**2)
    
    # function definitions
    fx = vx                        # dx/dt = vx
    fy = vy                        # dy/dt = vy
    fz = vz                        # dz/dt = vz
    fvx = -sign(vx) * rho*cd*area*v*vx/(2*m) + magnusx(vy, vz)/m     # dvx/dt = ax
    fvy = -sign(vy) * rho*cd*area*v*vy/(2*m) - g + magnusy(vx, vz)/m # dvydt = ay
    fvz = -sign(vz) * rho*cd*area*v*vz/(2*m) + magnusz(vx, vy)/m     # dvzdt = az
    
    return array([fx,fy,fz,fvx,fvy,fvz], float)

# This function solves df/dt = f(r,t) for Airdrag only
def fD(r,t):
    
    # unpack variables
    x = r[0]
    y = r[1]
    vx = r[2]
    vy = r[3]
    v = sqrt(vx**2 + vy**2)
    
    # function definitions
    fx = vx                        # dx/dt = vx
    fy = vy                        # dy/dt = vy
    fvx = -rho*cd*area*v*vx/(2*m)      # dvx/dt = ax
    fvy = -rho*cd*area*v*vy/(2*m) - g  # dvydt = ay
    
    return array([fx,fy,fvx,fvy], float)

# These functions calculate the new position for trajectory in vacuum
def x(t):
    return vx0*t + x0

def y(t):
    return -0.5*g*t**2 + vy0*t + y0

# Define initial values & constants here
x0 = 0 # initial x position
y0 = 0 # initial y position
z0 = 0 # initial z position
vx0 = 50 # x-component of the initial velocity
vy0 = 30 # y-component of the initial velocity
vz0 = 0 # z-component of the initial velocity
g = 9.8 # gravitational acceleration constant
cd = 0.32 # drag coefficient for baseball
rho = 1.225 # density of the medium (in this case, air)
rad = 0.0315 # radius of the baseball
area = pi*rad**2 # reference area (in this case, the reference area of a baseball)
m = 0.145 # mass of the ball
dt = 0.1 # timestep
wx = 0 # angular velocity x component
wy = 0 # angular velocity y component
wz = 40 # angular velocity z component
S = 0.17 # Magnus coefficient

t = 0 # elapsed time

# Magnus force simulation

vel1 = vector(vx0, vy0, 0) # velocity for Magnus force
posx1 = x0 # x position for Magnus force
posy1 = y0 # y position for Magnus force
posz1 = z0 # z position for Magnus force
r1 = array([x0, y0, z0, vx0, vy0, vz0], float) # r array for storing derivate results for Magnus force

# Airdrag simulation

vel2 = vector(vx0, vy0, 0) # velocity for air drag only
posx2 = x0 # x position for air drag only
posy2 = y0 # y position for air drag only
posz2 = z0 # z position for air drag only
r2 = array([x0, y0, vx0, vy0], float) # r array for storing derivate results for air drag only

# Classic Parabolic Motion

velx3 = vx0 # x-component of the velocity for trajectory in vacuum
vely3 = vy0 # y-component of the velocity for trajectory in vacuum
posx3 = x0 # x position for trajectory in vacuum
posy3 = y0 # y position for trajectory in vacuum

# Create the scene
scene = canvas(background = color.white, center=vector(0, 0, 0))
ball1 = sphere(pos=vector(posx1, posy1, 0), radius=rad, color=color.blue, make_trail=True) # Magnus force
ball2 = sphere(pos=vector(posx1, posy1, 0), radius=rad, color=color.red, make_trail=True) # Air drag only
ball3 = sphere(pos=vector(posx1, posy1, 0), radius=rad, color=color.green, make_trail=True) # Vacuum

# Graphs

# Height versus Time
motionY = graph(xtitle="time (sec)", ytitle="height (m)")

y1dots = gdots(graph= motionY, color=color.blue, label="Air Drag + Magnus", interval=1)
y2dots = gdots(graph= motionY, color=color.red, label="Air Drag Only", interval=1)
y3dots = gdots(graph= motionY, color=color.green, label="Classic Projectile", interval=1)

# Distance versus Time
motionX = graph(xtitle="time (sec)", ytitle="distance (m)")

x1dots = gdots(graph= motionX, color=color.blue, label="Air Drag + Magnus", interval=1)
x2dots = gdots(graph= motionX, color=color.red, label="Air Drag Only", interval=1)
x3dots = gdots(graph= motionX, color=color.green, label="Classic Projectile", interval=1)

# Trajectory graph
trajectory = graph(xtitle="distance (m)", ytitle="height (m)")

pos1dots = gdots(graph= trajectory, color=color.blue, label="Air Drag + Magnus", interval=1)
pos2dots = gdots(graph= trajectory, color=color.red, label="Air Drag Only", interval=1)
pos3dots = gdots(graph= trajectory, color=color.green, label="Classic Projectile", interval=1)

isBall1Done = False # Has the ball for Magnus force reached the ground?
isBall2Done = False # Has the ball for Air drag only reached the ground?
isBall3Done = False # Has the ball for trajectory in vacuum reached the ground?

# Run the program while there's at least one ball still in the air
while not (isBall1Done and isBall2Done and isBall3Done):
    
    rate(20)
    
    # Check if the plotting should terminate for magnus force motion
    if (not isBall1Done):
        # update the graphs
        y1dots.plot(t, r1[1])             
        x1dots.plot(t, r1[0])
        pos1dots.plot(r1[0], r1[1])
        # apply 4th Order Runge-Kutta
        k1 = dt*fM(r1,t)
        k2 = dt*fM(r1 + 0.5*k1, t + 0.5*dt)
        k3 = dt*fM(r1 + 0.5*k2, t + 0.5*dt)
        k4 = dt*fM(r1 + k3, t + dt)
        r1 += (k1 + 2*k2 + 2*k3 + k4)/6
    
    # Check if the plotting should terminate for air drag only motion
    if (not isBall2Done):
        # update the graphs
        y2dots.plot(t, r2[1])            
        x2dots.plot(t, r2[0])
        pos2dots.plot(r2[0], r2[1])
        # apply 4th Order Runge-Kutta
        k1 = dt*fD(r2,t)
        k2 = dt*fD(r2 + 0.5*k1, t + 0.5*dt)
        k3 = dt*fD(r2 + 0.5*k2, t + 0.5*dt)
        k4 = dt*fD(r2 + k3, t + dt)
        r2 += (k1 + 2*k2 + 2*k3 + k4)/6
    
    # Check if the plotting should terminate for classic parabolic motion in vacuum
    if (not isBall3Done):
        # update the graphs
        y3dots.plot(t, posy3)
        x3dots.plot(t, posx3)
        pos3dots.plot(posx3, posy3)
        # update the positions
        posx3 = x(t)
        posy3 = y(t)
    
    # update elapsed time
    t += dt
    
    # Update the S coefficient
    S = rad*math.sqrt(wx * wx + wy * wy + wz * wz)/ math.sqrt(vel1.x * vel1.x + vel1.y * vel1.y + vel1.z * vel1.z)
    
    # Update each ball's position
    ball1.pos = vector(r1[0], r1[1], r1[2])
    ball2.pos = vector(r2[0], r2[1], 0)
    ball3.pos = vector(posx3, posy3, 0)
    
    # Check for termination condition
    if (r1[1] < 0): isBall1Done = True
    if (r2[1] < 0): isBall2Done = True
    if (posy3 < 0): isBall3Done = True

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

This program combines the three programs (Classic Parabolic Motion, Air Drag, Magnus Force) into one single program and plots the data (heights versus time, distance traveled along x versus time, trajectory) into one graph. As in the Magnus Force program, in each while loop iteration, the new position of the baseball is calculated using the Runge-Kutta method and the Magnus coefficient is updated by using the Magnus force formula. For this program, I used a baseball with mass = 0.145 kg, radius = 0.0315 m, drag coefficient = 0.32. I used a variety of initial velocities and angular velocities to track down different behaviors of the baseball when it is thrown from the initial position (0, 0, 0). The simulation takes place in the air, which makes the rho = 1.225 and gravitational acceleration = 9.8 m/s^2.